In [3]:
import pandas as pd
import altair as alt
import numpy as np

In [4]:
def extrae_drive(url):
    edit = '/edit#gid='
    export ='/export?format=xlsx&gid='
    url = (url).replace(edit, export)
    return url

## Generando la Base principal
* Se usara la base de **cesados** y la base **Planilla**, esta sera filtrada segun las condiciones de la conclusion
* En esta nueva base se empezara a agregar el resto de informacion para el proyecto

In [5]:
# Extrae Cesados y planilla
cesados = extrae_drive('https://docs.google.com/spreadsheets/d/1bO_T0lAQgjcpjFFNVAwiXd2obwRjJS3K/edit#gid=2050623625')
cesados = pd.read_excel(
                cesados,
                usecols='E,I,O,P,S,T,W,X'
            ).rename(
                columns = {
                    'Total o Temprana':'TIEMPO_CESE',
                    'Producto':'PRODUCTO',
                    'TIPO':'TIPO_CESE',
                    'MOTIVO PRIMARIO':'MOTIVO_CESE'}
            )

planilla = extrae_drive('https://docs.google.com/spreadsheets/d/1L094YBwj3BvobbXb4t3ahzpmL6gBSxgs/edit#gid=648976010')
planilla = pd.read_excel(
                planilla,
                usecols='A,B,D:H'
            ).rename(
                columns={
                    'COD':'CODIGO',
                    'REGION/DPTO':'REGION',
                    'DIVISION/GC':'DIVISION'    }
            )

* Al aprecer hay colaboradores que cesaron mas de dos veces, es decir que ingresaron, cesaron, volvieron a ingresar y volvieron a cesar
* Se tomara en cuenta este numero de ceses como una caracteristca (aunque son pocas las personas) ***Revisar**
* Eliminare los colaboradores duplicados considerando solo los ultimos ceses, pues son de los que se tendria data mas reciente
* Genero la columna antiguedad definida como CESE menos INGRESO en meses

In [6]:
# Colaboradores con mas de 2 ceses
cesados.groupby(by = 'CODIGO', as_index=False).agg(n = ('CODIGO','count')).query('n>=2')

,CODIGO,n
425,A22AAAAA34,2
535,A33AAAA149,2
580,A33AAAA21A,2
642,A34AAAAA55,2
688,A36AAAAA18,2
1151,A4AAAAA57A,2
1212,A4AAAAA742,2
1397,A57AAAAA17,2
1408,A57AAAAA37,2
1938,A84AAAAA56,2


In [7]:
cesados = cesados.sort_values(
    by = 'CESE',
    ascending = False # de mayor a menor
).drop_duplicates(
    'CODIGO',
    keep = 'first' # los mas recientes
).iloc[:, [0,2,3,6,7]] #solo selecciono las columnas que usare

In [8]:
cesados['antiguedad_meses'] = (((cesados['CESE'] - cesados['INGRESO']).dt.days)/30).round(0) # Creo antiguedad

* Utilizo como base Planilla que tiene un registro de colabores cesados y activos
* Elimino los duplicados pues, la planilla cuenta con registro de colaboradores por meses, me quedo solo con los mas recientes
* Realizo los filtros para generar la base que quiero

In [9]:
planilla = planilla.drop_duplicates(
    'CODIGO',
    keep = 'last').iloc[:, 1:6] # no me interesa la fecha, solo los valores unicos del codigo

In [10]:
df = pd.merge(
    planilla,
    cesados,
    on = 'CODIGO',
    how = 'left'
).query('PRODUCTO == "GRUPAL"')

* Genero una base para Grupal y Otra para invidual porque seran dos diferentes proyectos con caracteristicas diferentes
* Empiezo a filtrar, Cargo, Tipo de cese, motivo y antiguedad
    * Como uni la base de ceses con planilla se han generado vacios de los colaboradores activos, para ellos debo de tener cuidado en los filtros

In [11]:
df = df[
    (df['CARGO'] == 'VENTA GRUPAL')
    & (df['TIPO_CESE'] != 'INVOLUNTARIA') #vacio o voluntaria
    & (~df['MOTIVO_CESE'].isin(['Personal Eventual', 'No le Gusto el Trabajo'])) # los otros motivos o vacios
    & ((df['antiguedad_meses'] >= 9) # quito los mayores a 9 meses
    | (df['antiguedad_meses'].isna()))
    ] # o que sean activos

* Creo una nueva columna que clasifique si es activo o cesado, usando el TIPO_CESE como referencia
* Reemplazo la fecha de los que no cesaron con la fecha de cierre, es decir el 01/01/2022
* Selecciono solo las variables que usare:
    * Ya no usare INGRESO porque necesito extraerno de otra base donde tenga el ingreso de los activos y cesados
    * Ya no usare TIPO_CESE,  MOTIVO_CESE ya no competen en el analisis
    * Ya no usare antiguedad_meses, pues generare una nueva

In [12]:
df.loc[df['TIPO_CESE'] == 'VOLUNTARIA', 'Estado'] = 'CESADO'
df.loc[df['TIPO_CESE'] != 'VOLUNTARIA', 'Estado'] = 'ACTIVO'
df.loc[df['CESE'].isna(), 'CESE'] = '2022-01-01'

df = df.iloc[:,[0,1,2,3,4,6,10]]

* Esta es la base que usare como principal donde ire agregando la informacion de otras variables

In [13]:
df.isna().sum() #No tengo NAS hasta el mento

CODIGO      0
CARGO       0
AGENCIA     0
PRODUCTO    0
REGION      0
CESE        0
Estado      0
dtype: int64

* Es necesario ver si la base con la que vamos a continuar esta balanceada
    *  Como se muestra tenemos **30% de Cesados y 70% de Activos**, por lo que podemos continuar con el analisis

In [14]:
df.groupby(
    by = 'Estado',
    as_index = False
).agg(
    n = ('CODIGO' , 'count')
)['n'] / df.shape[0]

0    0.641563
1    0.358437
Name: n, dtype: float64

## Agregando variables a la base principal

### Datos personales
* Una tabla con caracteristicias principales e importantes de los colaboradores
    * Como se vio en los casos anteriores hay colaboradores con varias fechas de ingreso, es necesario hacer una limpieza de esto considerando solo los ultimos ingresos
    * Cambio a inicio de mes la fecha de ingreso para tener un estandar

In [15]:
datos_personales = extrae_drive('https://docs.google.com/spreadsheets/d/1ku5CfOhQBpML0gXkuRIIEDjqG0liPdKt/edit#gid=254224400')
datos_personales = pd.read_excel(
                        datos_personales,
                        usecols = 'A, C:G'
                    ).sort_values(
                        by = 'INGRESO',
                        ascending = False
                    ).drop_duplicates(
                        'CODIGO',
                        keep = 'first')

## Obteniendo la fecha inicial del mes en el ingreso    
datos_personales['INGRESO'] = datos_personales['INGRESO'].to_numpy().astype('datetime64[M]')

* Uno la tabla de los datos personales con el dataframe que se tiene hasta el momento
* Hallo la antiguedad con el nuevo cruce y la edad en el cese


In [16]:
df = pd.merge(
        df,
        datos_personales,
        on = 'CODIGO',
        how = 'left')

In [17]:
# Hallando la antiguedad en la Fecha de cese y fecha ingreso
df['antiguedad'] = (((df['CESE'] - df['INGRESO']).dt.days)/30).round(0) #meses
df['edad'] = (((df['CESE'] - df['NACIMIENTO']).dt.days)/360).round(0) #anios

* Encontre negativos en la diferencia de edad, esto debido a que hay personas que tienen una fecha de cese que es de su primer ingreso, y una fecha de segundo ingreso, pero a la fecha no cesan por segunda vez. Para solucionar esto, debo reemplazar la fecha cese por la fecha de cierre que estoy tomando (enero 2021) y pasar de Estado CESADO a ACTIVO.
* Recalculo la columna antiguedad  para tener las nuevas antiguedades con los cambios de fecha


In [18]:
df.query('antiguedad < 0').sort_values(by = 'antiguedad')#['CODIGO'].unique()

,CODIGO,CARGO,AGENCIA,PRODUCTO,REGION,CESE,Estado,INGRESO,NACIMIENTO,SEXO,Educacion,EC,antiguedad,edad
117,A33AAAA163,VENTA GRUPAL,OFI CUSCO,GRUPAL,TERR SUR 2,2019-01-19,CESADO,2020-06-01,1993-11-21,F,GRADO DE BACHILLER,SOLTERO/A,-17.0,26.0
1376,AA8AAA2878,VENTA GRUPAL,OFI HUAYCAN,GRUPAL,TERR LIMA ESTE,2017-10-02,CESADO,2019-03-01,1994-10-07,F,TITULADO,CONCUBINATO / CONVIVIENTE,-17.0,23.0
1831,A11AAAAAA3,VENTA GRUPAL,OFI VENTANILLA,GRUPAL,TERR NORTE CHICO,2019-02-12,CESADO,2020-03-01,1993-08-20,F,EDUCACIÓN UNIVERSITARIA INCOMPLETA,SOLTERO/A,-13.0,26.0
1155,A4AAAAA135,VENTA GRUPAL,OFI PIURA,GRUPAL,TERR PIURA,2017-06-01,CESADO,2018-06-01,1993-04-01,F,GRADO DE BACHILLER,SOLTERO/A,-12.0,25.0
1402,AA7AAA5182,VENTA GRUPAL,OFI SANTA ROSA,GRUPAL,TERR SUR 2,2018-04-17,CESADO,2019-04-01,1992-08-17,F,EDUCACIÓN UNIVERSITARIA COMPLETA,SOLTERO/A,-12.0,26.0
1778,AA7AAA3443,VENTA GRUPAL,OFI CHINCHA,GRUPAL,TERR SUR CHICO,2018-12-31,CESADO,2020-01-01,1992-01-10,F,EDUCACIÓN UNIVERSITARIA COMPLETA,SOLTERO/A,-12.0,27.0
1830,A68AAAAA12,VENTA GRUPAL,OFI CASTILLA,GRUPAL,TERR PIURA,2019-03-31,CESADO,2020-03-01,1995-03-22,F,EDUCACIÓN UNIVERSITARIA COMPLETA,SOLTERO/A,-11.0,24.0


In [19]:
cod = df.query('antiguedad < 0')['CODIGO'].unique()
df.loc[df['CODIGO'].isin(cod), 'CESE'] = '2021-01-01'
df.loc[df['CODIGO'].isin(cod), 'Estado'] = 'ACTIVO'
df['antiguedad'] = (((df['CESE'] - df['INGRESO']).dt.days)/30).round(0) #meses

### Bonos de Venta Grupal
* Este valor diferencia el ingreso extra que tiene un colaborador de otro respecto a su desempeno, lo llamaremos RV por Remuneracion Variable
* Tomare los valores promedio que genero un colaborador desde que esta en la empresa, pero con unas consideraciones
    * Es importante tomar en cuenta el periodo covid donde las ventas y por lo tanto la remuneracion variable es atipica


In [20]:
bonos_venta = extrae_drive('https://docs.google.com/spreadsheets/d/1POTE8j5SneZ0QSvmbtKL_t4hu2odWlTM/edit#gid=578896078')
bonos_venta = pd.read_excel(
                    bonos_venta,
                    usecols = 'A, B, CE'
                ).rename(
                    columns = {
                    'Codigo Trabajador': 'CODIGO',
                    'RV TOTAL': 'RV'}
                )

* Como se ve, los periodos de marzo 2020 a octubre 2020 (inclusive) tienen valores muy bajos debido a la pandemia, por lo que para este analisis no lo tomaremos en cuenta, pues nos bajaria el promedio atipicamente

In [21]:
rv_agrup = bonos_venta.groupby(
                by = 'MES',
                as_index = False
            ).agg(
                rv_total = ('RV' , 'sum')
            )

principal = alt.Chart(
    rv_agrup
).mark_line(
    color = '#04328C'
).encode(
    x = alt.X(
        'MES:T',
        title = None),
    y = alt.Y(
        'rv_total:Q',
        title = None,
        axis = None)
)

linea_i = alt.Chart(
    pd.DataFrame({'MES': ['2020-03-01']})
).mark_rule(
    color = 'red',
    strokeWidth = 2
).encode(
    x = 'MES:T')

linea_f = alt.Chart(
    pd.DataFrame({'MES': ['2020-10-01']})
).mark_rule(
    color = 'red',
    strokeWidth = 2
).encode(
    x = 'MES:T')


(principal + linea_i + linea_f).properties(
    title = 'RV total Historico',
    width = 700,
    height = 300
).configure_view(
    strokeWidth = 0
 ).configure_axis(
    labelFontSize = 13,
    labelAngle = 0
 ).configure_title(
    fontSize = 18,
    font = 'courier',
    color = '#033E8C',
    anchor = 'start'
)

alt.LayerChart(...)

* Creando la nueva base segun los filtro considerados
* Utilizaremos el rv promedio y la media pues estos varian mucho en algunos casos

In [22]:
bonos_venta = bonos_venta.query(
                'MES <= "2020-03-01" | MES >= "2020-10-01" '
            ).groupby(
                by = 'CODIGO',
                as_index = False
            ).agg(
                rv_prom = ('RV', 'mean'),
                rv_median = ('RV', 'median')
            )

In [23]:
bonos_venta.describe

<bound method NDFrame.describe of           CODIGO      rv_prom    rv_median
0     111AAAAA11  2180.899091  1881.308000
1     111AAAAA12  1991.281653  1779.360000
2     111AAAAA13  2147.702064  1337.237000
3     111AAAAA14  2008.110987  1297.600000
4     111AAAAA15  2312.259572  1710.335000
...          ...          ...          ...
3069  AA9AAAAAA4  2956.422095  1040.000000
3070  AA9AAAAAA5  3205.058000  2279.300000
3071  AA9AAAAAA6  4098.077304  1181.928000
3072  AA9AAAAAA7  2258.831670  1620.294288
3073  AA9AAAAAA9  4319.952185  2519.460000

[3074 rows x 3 columns]>

In [24]:
df = pd.merge(
        df,
        bonos_venta,
        on = 'CODIGO',
        how = 'left')

* Se han encontrado 13 colaboradores que no estan en la base del RV, al parecer porque son reciente y no han generado un bono extra
* Debido a que son pocos, se reemplazaran los null con el promedio del RV total

In [25]:
df['rv_prom'] = df['rv_prom'].fillna(bonos_venta['rv_prom'].mean())
df['rv_median'] = df['rv_median'].fillna(bonos_venta['rv_median'].mean())

In [26]:
df.isna().sum()

CODIGO        0
CARGO         0
AGENCIA       0
PRODUCTO      0
REGION        0
CESE          0
Estado        0
INGRESO       0
NACIMIENTO    0
SEXO          0
Educacion     0
EC            0
antiguedad    0
edad          0
rv_prom       0
rv_median     0
dtype: int64

### Programa de Mejora
* Analizando los colaboradores que estuvieron en el programa de mejora

In [38]:
programa_mejora = extrae_drive('https://docs.google.com/spreadsheets/d/1CKFlLpx248yvZS0INAawAA5e8NC2qdvV/edit#gid=1510069928')
programa_mejora = pd.read_excel(
                    programa_mejora,
                    usecols = 'C, F, Q'
                ).rename(
                    columns = {
                    'CODIGO CFIS': 'CODIGO',
                    'MOTIVO PRINCIPAL': 'MOTIVO_PM'}
                ).query(
                    'Cargo == "VENTA GRUPAL"'
                )

1. Hallo el numero de veces que el colaborador estuvo en el programa de mejoras, porque al parecer hay algunos colaboradores que tiene  mas de 3 veces en el programa
2. El numero de veces que estuvo en el progrma de mejora segun el motivio

In [39]:
pm_total = programa_mejora.groupby(
                    by='CODIGO', as_index=False
                ).agg(pm_total = ('CODIGO','count'))

In [44]:
pm_motivos = pd.get_dummies(
                programa_mejora,
                columns = ['MOTIVO_PM']
            ).groupby(
                by = 'CODIGO', as_index = False
            ).agg(
                MOTIVO_PM_DESEMP = ('MOTIVO_PM_DESEMPEÑO','sum'),
                MOTIVO_PM_DISCIPLINA = ('MOTIVO_PM_DISCIPLINA','sum')
            )

In [42]:
df = pd.merge(
        df,
        pm_total,
        on = 'CODIGO',
        how = 'left')

In [45]:
df = pd.merge(
        df,
        pm_motivos,
        on = 'CODIGO',
        how = 'left')

In [46]:
df.sample(10)

,CODIGO,CARGO,AGENCIA,PRODUCTO,REGION,CESE,Estado,INGRESO,NACIMIENTO,SEXO,Educacion,EC,antiguedad,edad,rv_prom,rv_median,pm_total,MOTIVO_PM_DESEMP,MOTIVO_PM_DISCIPLINA
695,AA8AAA1936,VENTA GRUPAL,OFI COLLIQUE,GRUPAL,TERR LIMA NORTE,2021-09-10,CESADO,2015-07-01,1994-05-20,M,EDUCACIÓN UNIVERSITARIA COMPLETA,SOLTERO/A,75.0,28.0,8217.363754,5405.3120,NaN,NaN,NaN
956,A48AAAAAA4,VENTA GRUPAL,OFI PIURA,GRUPAL,TERR PIURA,2022-01-01,ACTIVO,2017-04-01,1984-06-06,F,EDUCACIÓN UNIVERSITARIA COMPLETA,SOLTERO/A,58.0,38.0,6108.417244,4307.6100,NaN,NaN,NaN
648,A79AAAA122,VENTA GRUPAL,OFI VILLA EL SALVADOR,GRUPAL,TERR LIMA SUR,2021-05-20,CESADO,2020-08-01,1998-11-24,F,EDUCACIÓN TÉCNICA INCOMPLETA,SOLTERO/A,10.0,23.0,3629.027800,3501.1744,NaN,NaN,NaN
1194,A13AAAAA31,VENTA GRUPAL,OFI JICAMARCA,GRUPAL,TERR SAN JUAN,2022-01-01,ACTIVO,2018-08-01,1990-05-23,M,EDUCACIÓN UNIVERSITARIA INCOMPLETA,CONCUBINATO / CONVIVIENTE,42.0,32.0,7311.245903,7014.4000,NaN,NaN,NaN
1626,A81AAAAA99,VENTA GRUPAL,OFI SAN JUAN DE MIRAFLORES,GRUPAL,TERR LIMA SUR,2022-01-01,ACTIVO,2019-09-01,1995-04-04,M,EDUCACIÓN UNIVERSITARIA INCOMPLETA,SOLTERO/A,28.0,27.0,4813.043708,5583.1820,NaN,NaN,NaN
1448,A48AAAAA49,VENTA GRUPAL,OFI PIURA,GRUPAL,TERR PIURA,2022-01-01,ACTIVO,2019-06-01,1992-05-15,M,EDUCACIÓN UNIVERSITARIA COMPLETA,SOLTERO/A,32.0,30.0,7592.693912,8528.0800,NaN,NaN,NaN
1666,A48AAAAA63,VENTA GRUPAL,OFI PIURA,GRUPAL,TERR PIURA,2022-01-01,ACTIVO,2019-10-01,1986-10-06,M,EDUCACIÓN UNIVERSITARIA INCOMPLETA,SOLTERO/A,27.0,36.0,6522.171109,5093.1100,NaN,NaN,NaN
780,AA7AAA4A82,VENTA GRUPAL,OFI TRUJILLO CENTRO,GRUPAL,TERR LA LIBERTAD,2022-01-01,ACTIVO,2015-07-01,1991-07-18,M,TITULADO,SOLTERO/A,79.0,31.0,4770.057971,3211.2000,1.0,1.0,0.0
650,1A3AAAAA64,VENTA GRUPAL,OFI TARAPOTO,GRUPAL,TERR ORIENTE,2021-05-31,CESADO,2020-09-01,1986-11-29,M,EDUCACIÓN UNIVERSITARIA INCOMPLETA,SOLTERO/A,9.0,35.0,2961.268400,2218.4000,NaN,NaN,NaN
392,A91AAAAAA6,VENTA GRUPAL,OFI HUARAL,GRUPAL,TERR NORTE CHICO,2020-02-03,CESADO,2018-06-01,1991-09-09,F,EDUCACIÓN UNIVERSITARIA INCOMPLETA,SOLTERO/A,20.0,29.0,4112.764800,1945.0000,NaN,NaN,NaN




* memos = extrae_drive('https://docs.google.com/spreadsheets/d/1Q2pvICQWLAh8Rvou9FvH085KULXioDVK/edit#gid=238204385')

vacaciones = extrae_drive('https://docs.google.com/spreadsheets/d/1riJOPN5KUwV1962wO6mjPL7PMgL2v3_s/edit#gid=800276956')
descanso_medico = extrae_drive('https://docs.google.com/spreadsheets/d/1H_oENDjS0FyDNVmSg1199PrE6s8q6z7-/edit#gid=1138295584')
bonos_venta_i = extrae_drive('https://docs.google.com/spreadsheets/d/1Vjh5S86c1r60L-CnuQB_m6JZPy223Z4p/edit#gid=100516943834575165411')

desempeno = extrae_drive()
reconocimientos = extrae_drive()
* hijxs = extrae_drive()

recategorizaciones = extrae_drive()
cobertura_ofi = extrae_drive()
* antg_oficina = extrae_drive()
* gptw_oficina = extrae_drive()